In [1]:
from lcatools import enum
from antelope_catalog import LcCatalog

In [2]:
cat = LcCatalog('/data/GitHub/2018/bw2-uncertainty/catalog/')

Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/flowable_synlist.json:
Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/quantity_synlist.json:
local.qdb: /data/GitHub/2018/bw2-uncertainty/catalog/reference-quantities.json
Loading JSON data from /data/GitHub/2018/bw2-uncertainty/catalog/reference-quantities.json:
local.qdb: /data/GitHub/lca-tools/lcatools/providers/data/elcd_reference_quantities.json
24 new quantity entities added (24 total)
5 new flow entities added (5 total)


In [4]:
cat.show_interfaces()

local.ecoinvent.3.4.apos [basic, index, inventory]
local.ecoinvent.3.4.apos.index__20180905 [background, basic, index]
local.ecoinvent.3.4.conseq [basic, index, inventory]
local.ecoinvent.3.4.cutoff [basic, index, inventory]
local.ecoinvent.3.4.cutoff.index__20180905 [background, basic, index]
local.lcia.ecoinvent.3.1.2014_08_13 [basic, index, quantity]
local.qdb [basic, index, quantity]


In [5]:
qe = cat.query('local.ecoinvent.3.4.cutoff')

### Utilization of truck datasets
Here I want to audit the way the Ecoinvent database uses truck transport internally.

Approach: 
 1. Identify all truck transport processes AND markets, call it T
 2. For each, identify consumers that are not in T
 3. Count 'em up
 
 

In [18]:
# 1- does searching on 'transport, freight, lorry' produce all truck transport processes?
t_f_l = [t for t in qe.processes(Classifications='4923')]

In [19]:
len(t_f_l)

150

In [20]:
m_t_f_l = [m for m in t_f_l if m['Name'].startswith('market')]

In [21]:
len(m_t_f_l)

55

In [22]:
u_t_f_l = [u for u in t_f_l if u not in m_t_f_l]

In [23]:
len(u_t_f_l)

95

In [24]:
p = u_t_f_l[0]

In [25]:
p.show()

ProcessRef catalog reference (0785e6ba-797d-48a3-9bce-32c315d247b0)
origin: local.ecoinvent.3.4.cutoff
UUID: 0785e6ba-797d-48a3-9bce-32c315d247b0
   Name: transport, freight, lorry 16-32 metric ton, EURO6
Comment: This dataset represents the service of 1tkm freight transport in a lorry of the size class 16-32 metric tons gross vehicle weight (GVW) and Euro VI emissions class. The transport datasets refer to the entire transport life cycle i.e. to the construction, operation, maintenance and end of life of vehicle and road infrastructures. Fuel consumption and emissions are for average European journeys and load factors and not representative of a specific transport scenario. The average load factors are taken from the Tremove model v2.7b (2009) and EcoTransIT (2011) report. These are as follows:

Lorry size class     Average load factor            GVW 

3.5-7.5t                             0.98t	                            4.98t 
7.5-16t                               3.29t             

In [77]:
all_uses = {p: [t for t in qe.consumers(p.external_ref)] for p in u_t_f_l}

In [78]:
_non_mkt_int = {p: [t for t in k if t.termination not in t_f_l] for p, k in all_uses.items()}

In [80]:
non_mkt_uses = {i.external_ref: j for i, j in _non_mkt_int.items() if len(j) > 0}

In [82]:
# exactly one truck transport process used directly, i.e. not by another truck process
non_mkt_uses

{'e0c10b7b-38c5-4b6f-9900-d72933c3d3c1': [ProductFlow(origin='local.ecoinvent.3.4.cutoff.index__20180905', flow=<lcatools.entity_refs.flow_ref.FlowRef object at 0x7fe1385f3128>, direction='Output', termination=<lcatools.entity_refs.process_ref.ProcessRef object at 0x7fe117aff6d8>, component_id='00053b25-15e5-498c-bc77-4850ea3475ca')]}

In [83]:
# here it is
non_mkt_uses['e0c10b7b-38c5-4b6f-9900-d72933c3d3c1'][0].termination.show()

ProcessRef catalog reference (058cd712-6a33-423e-a9c8-f9d69389cc07)
origin: local.ecoinvent.3.4.cutoff
UUID: 058cd712-6a33-423e-a9c8-f9d69389cc07
   Name: glass wool mat production, Saint-Gobain ISOVER SA
Comment: This dataset is provided by Saint-Gobain ISOVER SA, CH-1522 Lucens and represents a cradle-to-gate inventory of the average production of all glass woll mats in 2009 per kg. The average density of an St. Gobain ISOVER SA glass wool mat is 25.02 kg/m3.
The following packaging per m3 is not included: corrugated board, mixed fibre, single wall: 1.66E-03 kg/m3; paper, wood containing, LWC: 1.71E-04 kg/m3: packaging film, LDPE: 2.43E-02 kg/m3; EUR-flat pallet: 3.33E-03 pce/m3
More information on the glass wool by St. Gobain ISOVER SA and their technical specifications can be found under http://www.isover.ch/
==Local Fields==
           Name: glass wool mat production, Saint-Gobain ISOVER SA
   SpatialScope: CH
  TemporalScope: {'begin': '2009-01-01', 'end': '2017-12-31'}
Classific

In [84]:
# now, let's check utilization of the markets themselves
all_mkt_supplies = {m: [t for t in qe.consumers(m.external_ref)] for m in m_t_f_l}
_non_mkt_mkt = {m: [t for t in k if t.termination not in t_f_l] for m, k in all_mkt_supplies.items()}
intra_db_users = {i.external_ref: j for i, j in _non_mkt_mkt.items() if len(j) > 0}

In [85]:
len(all_mkt_supplies)

55

In [87]:
len(intra_db_users)

14

In [143]:
sorted_trucks = sorted(((len(k), qe.get(i)) for i, k in intra_db_users.items()), key=lambda x: x[0], reverse=True)

In [144]:
['%4d %s [%s]' % (i, k['Name'], k['SpatialScope']) for i, k in sorted_trucks]

['1902 market for transport, freight, lorry, unspecified [GLO]',
 ' 480 market for transport, freight, light commercial vehicle [GLO]',
 ' 282 market for transport, freight, lorry 7.5-16 metric ton, EURO3 [GLO]',
 '  44 market for transport, freight, lorry with reefer, cooling [GLO]',
 '  44 market for transport, freight, lorry with refrigeration machine, cooling [GLO]',
 '  27 market for transport, freight, lorry 16-32 metric ton, EURO5 [GLO]',
 '  10 market for transport, freight, lorry >32 metric ton, EURO4 [GLO]',
 '   9 market for transport, freight, lorry 16-32 metric ton, EURO3 [GLO]',
 '   8 market for transport, freight, lorry 16-32 metric ton, EURO4 [GLO]',
 '   4 market for transport, freight, lorry 16-32 metric ton, EURO6 [GLO]',
 '   2 market for transport, freight, lorry 3.5-7.5 metric ton, EURO4 [GLO]',
 '   2 market for transport, freight, lorry 3.5-7.5 metric ton, EURO5 [GLO]',
 '   2 market for transport, freight, lorry 3.5-7.5 metric ton, EURO3 [GLO]',
 '   2 market 

In [163]:
ps = sorted_trucks[0][1]

suppliers = enum(qe.get(x.termination) for x in ps.inventory(ps.reference()))
suppliers1 = enum(qe.get(x.termination) for x in suppliers[0].inventory(suppliers[0].reference()))


 [00] local.ecoinvent.3.4.cutoff/8bfedc32-d1af-449f-b25f-3e651056c84b transport, freight, lorry, all sizes, EURO3 to generic market for transport, freight, lorry, unspecified [RER]
 [01] local.ecoinvent.3.4.cutoff/b6efa12c-c545-409d-b359-783d3ca18c7b transport, freight, lorry, all sizes, EURO4 to generic market for transport, freight, lorry, unspecified [RER]
 [02] local.ecoinvent.3.4.cutoff/1e30a190-f561-410b-ab9a-256c9e1cce5f transport, freight, lorry, all sizes, EURO5 to generic market for transport, freight, lorry, unspecified [RER]
 [03] local.ecoinvent.3.4.cutoff/0884a875-86d6-412a-a22d-4bda41265ea6 transport, freight, lorry, all sizes, EURO6 to generic market for transport, freight, lorry, unspecified [RER]
 [04] local.ecoinvent.3.4.cutoff/e36de8ae-75c5-4451-bcbd-03826e02d9fa transport, freight, lorry, all sizes, EURO3 to generic market for transport, freight, lorry, unspecified [RoW]
 [05] local.ecoinvent.3.4.cutoff/4afa1d38-1f91-4283-b744-dd1d71c186bc transport, freight, lorry

In [138]:
from antelope_reports.tables.allocation import AllocationGrid

In [123]:
import importlib

In [131]:
import antelope_reports.tables

In [136]:
importlib.reload(antelope_reports.tables.allocation)

<module 'antelope_reports.tables.allocation' from '/data/GitHub/lca-tools/antelope_reports/tables/allocation.py'>

In [154]:
suppliers[1].show()

ProcessRef catalog reference (78d7c917-01ad-4d85-a268-70e4f47c0381)
origin: local.ecoinvent.3.4.cutoff
UUID: 78d7c917-01ad-4d85-a268-70e4f47c0381
   Name: transport, freight, lorry 7.5-16 metric ton, EURO3
Comment: This dataset represents the service of 1tkm freight transport in a lorry of the size class 7.5-16 metric tons gross vehicle weight (GVW) and Euro III emissions class. The transport datasets refer to the entire transport life cycle i.e. to the construction, operation, maintenance and end of life of vehicle and road infrastructures. Fuel consumption and emissions are for average European journeys and load factors and not representative of a specific transport scenario. The average load factors are taken from the Tremove model v2.7b (2009) and EcoTransIT (2011) report. These are as follows:

Lorry size class     Average load factor            GVW 

3.5-7.5t                             0.98t	                            4.98t 
7.5-16t                               3.29t          

In [166]:
suppliers2 = enum(qe.get(x.termination) for x in suppliers1[0].inventory(suppliers1[0].reference()))
suppliers22 = enum(qe.get(x.termination) for x in suppliers1[2].inventory(suppliers1[2].reference()))

 [00] local.ecoinvent.3.4.cutoff/0dfba58f-6ea2-4d5f-9bfc-8fbffa337580 transport, freight, lorry 16-32 metric ton, EURO3 [RER]
 [01] local.ecoinvent.3.4.cutoff/6b5c2624-62ac-40f8-8145-b66efb4f189f transport, freight, lorry 16-32 metric ton, EURO3 [RoW]
 [00] local.ecoinvent.3.4.cutoff/7ebc59ca-9062-4643-b467-72db5589c34a transport, freight, lorry 7.5-16 metric ton, EURO3 [RER]
 [01] local.ecoinvent.3.4.cutoff/78d7c917-01ad-4d85-a268-70e4f47c0381 transport, freight, lorry 7.5-16 metric ton, EURO3 [RoW]


In [167]:
AllocationGrid(cat.get_archive('local.ecoinvent.3.4.cutoff', 'inventory'), *suppliers2, *suppliers22, report_unallocated=False).text()

Direction Ref C0         C1         C2         C3                                                               
----------------------------------------------------------------------------------------------------------------
Output    {*}          1          1                       transport, freight, lorry 16-32 metric ton, EURO3     
""        {*}                                1          1 transport, freight, lorry 7.5-16 metric ton, EURO3    
Input           -0.00022   -0.00022  -0.000227  -0.000227 [GLO] market for tyre wear emissions, lorry           
                  0.0378                0.0481            [RER] market group for diesel, low-sulfur             
""                           0.0378                0.0481 [RoW] market for diesel, low-sulfur                   
""               3.2e-07    3.2e-07   5.63e-07   5.63e-07 [GLO] market for lorry, 16 metric ton                 
""             -2.22e-05  -2.22e-05  -2.29e-05  -2.29e-05 [GLO] market for brake wear emissions,

In [168]:
from antelope_reports.tables import FlowablesGrid

In [173]:
qe.get('349b29d1-3e58-4c66-98b9-9d1a076efd2e').show()

FlowRef catalog reference (349b29d1-3e58-4c66-98b9-9d1a076efd2e)
origin: local.ecoinvent.3.4.cutoff
UUID: 349b29d1-3e58-4c66-98b9-9d1a076efd2e
   Name: Carbon dioxide, fossil
Comment: EcoSpold02 Flow
==Local Fields==
       Name: Carbon dioxide, fossil
  CasNumber: 000124-38-9
Compartment: ['air', 'unspecified']
    Comment: EcoSpold02 Flow


In [171]:
FlowablesGrid(cat.qdb, suppliers2[0].inventory(suppliers2[0].reference())).text()

Flowable                 Compartment              C0                                                            
----------------------------------------------------------------------------------------------------------------
000050-00-0              air                        3.38e-06 [kg] formaldehyde                                  
000071-43-2              ""                         2.82e-08 [kg] benzene                                       
000074-82-8              ""                         9.89e-07 [kg] methane                                       
000074-84-0              ""                         1.21e-08 [kg] ethane                                        
000074-98-6              ""                         4.02e-08 [kg] propane                                       
000075-07-0              ""                         1.84e-06 [kg] acetaldehyde                                  
000095-47-6              ""                         1.61e-07 [kg] o-xylene                      

In [147]:
help(AllocationGrid)

Help on class AllocationGrid in module antelope_reports.tables.allocation:

class AllocationGrid(antelope_reports.tables.base.BaseTableOutput)
 |  This table can be produced dynamically
 |  
 |  Method resolution order:
 |      AllocationGrid
 |      antelope_reports.tables.base.BaseTableOutput
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, archive, *prefs, report_unallocated=True)
 |      :param archive:
 |      :param prefs: one or more process refs
 |      :param report_unallocated: [True] by default, report process's unallocated values alongside the allocated ones
 |  
 |  add_column(self, arg)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from antelope_reports.tables.base.BaseTableOutput:
 |  
 |  text(self, width=10, hdr_width=24, max_width=112, expanded=True)
 |      Outputs the table in text format
 |      :return: nothing.
 |  
 |  -----------------------------------------------------------

In [88]:
_gen_idbu = ((i, j) for i, j in intra_db_users.items())

In [103]:
ref, users = next(_gen_idbu)

In [104]:
qe.get(ref).show()

ProcessRef catalog reference (72f7eacb-4200-4a3f-a3c7-45d9ec551f0b)
origin: local.ecoinvent.3.4.cutoff
UUID: 72f7eacb-4200-4a3f-a3c7-45d9ec551f0b
   Name: market for transport, freight, lorry 16-32 metric ton, EURO4
Comment: no comment.
==Local Fields==
           Name: market for transport, freight, lorry 16-32 metric ton, EURO4
   SpatialScope: GLO
  TemporalScope: {'begin': '2011-01-01', 'end': '2017-12-31'}
Classifications: ['ISIC rev.4 ecoinvent: 4923:Freight transport by road', 'EcoSpold01Categories: transport systems/road']
        Comment: no comment.


In [105]:
len(users)

8

In [107]:
[str(u.termination) for u in users]

['local.ecoinvent.3.4.cutoff/9a7bdcef-8d6e-49f5-8794-d21c19b7c45b pipeline construction, liquid manure [CH]',
 'local.ecoinvent.3.4.cutoff/c36f9797-f21a-4704-bf56-ad0b14029000 pipeline construction, liquid manure [RoW]',
 'local.ecoinvent.3.4.cutoff/7e6eee3b-06b2-4110-b1f8-cc346ebb6cd5 market for tomato seedling, for planting [GLO]',
 'local.ecoinvent.3.4.cutoff/4329378d-7e2c-4230-8a38-9def4d649bc3 market for strawberry seedling, for planting [GLO]',
 'local.ecoinvent.3.4.cutoff/e4f7c598-3ee7-4c5f-a3dc-ad76bc280d84 market for waste polyethylene terephthalate, for recycling, unsorted [RoW]',
 'local.ecoinvent.3.4.cutoff/0afd8d6a-a6b9-485c-96f6-9ee93660323a market for waste polyethylene, for recycling, unsorted [CH]',
 'local.ecoinvent.3.4.cutoff/ccc53607-e044-4e13-b195-e5acba2ebc6d market for waste polyethylene terephthalate, for recycling, unsorted [CH]',
 'local.ecoinvent.3.4.cutoff/63be82de-5170-4905-9e42-1822d50c8e3f market for waste polyethylene, for recycling, unsorted [RoW]']

In [27]:
enum(t for t in qe.consumers(p.external_ref) if t.termination not in m_t_f_l)

[]